In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import prince

In [2]:
df = pd.read_csv('data/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2018.csv')
df.head()

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,Capital/Adirond,Columbia,1001000.0,146.0,Columbia Memorial Hospital,50 to 69,125,F,White,Not Span/Hispanic,...,Major,Major,Medical,"Managed Care, Unspecified",Self-Pay,NaN,NaN,True,17591.41,3570.06
1,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,Extreme,Extreme,Surgical,Medicare,Medicaid,NaN,NaN,True,106190.21,78913.24
2,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,False,5600.00,4161.53
3,New York City,Kings,7001016.0,1301.0,Kings County Hospital Center,18 to 29,112,M,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,Medicaid,NaN,NaN,True,15965.54,11341.30
4,New York City,Manhattan,7002054.0,1464.0,New York-Presbyterian Hospital - Columbia Pres...,0 to 17,104,M,Multi-racial,Not Span/Hispanic,...,Major,Moderate,Medical,Medicaid,Medicaid,Self-Pay,NaN,True,36846.83,10240.02


In [3]:
# Cleaning steps
# Drop any columns with > 30% null values
df_cleaned = df.drop(['Birth Weight', 'Payment Typology 3', 'Payment Typology 2', 'CCSR Procedure Code', 'CCSR Procedure Description'], axis=1)
df_cleaned.dropna(inplace=True)
df_cleaned = df_cleaned[df_cleaned['Length of Stay'] != '120 +']
df_cleaned

,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Emergency Department Indicator,Total Charges,Total Costs
0,Capital/Adirond,Columbia,1001000.0,146.0,Columbia Memorial Hospital,50 to 69,125,F,White,Not Span/Hispanic,...,1.0,DISEASES AND DISORDERS OF THE NERVOUS SYSTEM,3.0,Major,Major,Medical,"Managed Care, Unspecified",True,17591.41,3570.06
1,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,4.0,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,4.0,Extreme,Extreme,Surgical,Medicare,True,106190.21,78913.24
2,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,19.0,MENTAL DISEASES AND DISORDERS,2.0,Moderate,Minor,Medical,Medicaid,False,5600.00,4161.53
3,New York City,Kings,7001016.0,1301.0,Kings County Hospital Center,18 to 29,112,M,Black/African American,Not Span/Hispanic,...,5.0,DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,2.0,Moderate,Minor,Medical,Medicaid,True,15965.54,11341.30
4,New York City,Manhattan,7002054.0,1464.0,New York-Presbyterian Hospital - Columbia Pres...,0 to 17,104,M,Multi-racial,Not Span/Hispanic,...,4.0,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,3.0,Major,Moderate,Medical,Medicaid,True,36846.83,10240.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352802,New York City,Queens,7003001.0,1628.0,Flushing Hospital Medical Center,30 to 49,113,M,White,Not Span/Hispanic,...,20.0,ALCOHOL/DRUG USE AND ALCOHOL/DRUG INDUCED ORGA...,1.0,Minor,Minor,Medical,Medicaid,False,6764.00,1722.84
2352803,New York City,Kings,7001021.0,1306.0,New York - Presbyterian Brooklyn Methodist Hos...,70 or Older,112,M,Black/African American,Not Span/Hispanic,...,18.0,INFECTIOUS AND PARASITIC DISEASES (SYSTEMIC OR...,4.0,Extreme,Major,Medical,Medicare,True,43488.79,11414.60
2352804,New York City,Kings,7001021.0,1306.0,New York - Presbyterian Brooklyn Methodist Hos...,30 to 49,112,F,White,Spanish/Hispanic,...,11.0,DISEASES AND DISORDERS OF THE KIDNEY AND URINA...,2.0,Moderate,Moderate,Medical,Medicaid,True,86509.76,19997.25
2352805,New York City,Kings,7001021.0,1306.0,New York - Presbyterian Brooklyn Methodist Hos...,18 to 29,112,F,White,Not Span/Hispanic,...,14.0,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",1.0,Minor,Minor,Medical,Medicaid,True,21954.93,11657.60


In [4]:
#Predict Length of Stay
y = df_cleaned[['Length of Stay']]
X = df_cleaned.loc[:, ~df_cleaned.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]
print(y.columns)

Index(['Length of Stay'], dtype='object')


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

categorical_columns = X_train.select_dtypes(exclude='number')
print(categorical_columns.columns)

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train[column].replace(values, encoded_values, inplace=True)

# Splitting the data into training and testing data
'''regr = LinearRegression()
  
regr.fit(X_train, y_train)
print(regr.score(X_test, y_test))'''

Index(['Hospital Service Area', 'Hospital County', 'Facility Name',
       'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race', 'Ethnicity',
       'Type of Admission', 'Patient Disposition', 'CCSR Diagnosis Code',
       'CCSR Diagnosis Description', 'APR DRG Description',
       'APR MDC Description', 'APR Severity of Illness Description',
       'APR Risk of Mortality', 'APR Medical Surgical Description',
       'Payment Typology 1', 'Emergency Department Indicator'],
      dtype='object')


'regr = LinearRegression()\n  \nregr.fit(X_train, y_train)\nprint(regr.score(X_test, y_test))'

In [6]:
X_train.head()

,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Emergency Department Indicator
1433062,4,12,5957001.0,180.0,22,5,11,2,4,3,...,751.0,76,19.0,3,2.0,3,3,1,7,2
701162,6,11,2952005.0,552.0,128,1,30,1,1,1,...,222.0,93,6.0,9,4.0,2,1,2,2,1
126858,6,11,2951001.0,541.0,83,4,16,1,4,1,...,540.0,16,14.0,11,1.0,4,3,2,4,2
977838,1,16,2801001.0,484.0,49,2,10,1,1,1,...,560.0,19,14.0,11,2.0,3,3,1,3,2
1196429,2,6,7003004.0,1630.0,10,4,12,2,4,1,...,775.0,33,20.0,12,3.0,1,4,1,3,2


In [7]:
regr = LinearRegression()
  
regr.fit(X_train, y_train)
categorical_columns = X_test.select_dtypes(exclude='number')
print(categorical_columns.columns)

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_test[column].replace(values, encoded_values, inplace=True)
preds = regr.predict(X_test)
mae = metrics.mean_absolute_error(y_test, preds)
mse = metrics.mean_squared_error(y_test, preds)
r2 = metrics.r2_score(y_test, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

Index(['Hospital Service Area', 'Hospital County', 'Facility Name',
       'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race', 'Ethnicity',
       'Type of Admission', 'Patient Disposition', 'CCSR Diagnosis Code',
       'CCSR Diagnosis Description', 'APR DRG Description',
       'APR MDC Description', 'APR Severity of Illness Description',
       'APR Risk of Mortality', 'APR Medical Surgical Description',
       'Payment Typology 1', 'Emergency Department Indicator'],
      dtype='object')
The model performance for testing set
--------------------------------------
MAE is 3.3569129476012254
MSE is 41.20639938906745
R2 score is 0.21030286352127303


In [8]:
#Manual feature selection based on correlations
y = df_cleaned[['Length of Stay']]
X = df_cleaned[['Age Group', 'Type of Admission', 'Patient Disposition', 'APR Severity of Illness Code', 'APR Severity of Illness Description', 'APR Risk of Mortality', 'Emergency Department Indicator']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

categorical_columns = X_train.select_dtypes(exclude='number')
print(categorical_columns.columns)

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train[column].replace(values, encoded_values, inplace=True)

Index(['Age Group', 'Type of Admission', 'Patient Disposition',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'Emergency Department Indicator'],
      dtype='object')


In [9]:
regr = LinearRegression()
  
regr.fit(X_train, y_train)
categorical_columns = X_test.select_dtypes(exclude='number')
print(categorical_columns.columns)

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_test[column].replace(values, encoded_values, inplace=True)
preds = regr.predict(X_test)
mae = metrics.mean_absolute_error(y_test, preds)
mse = metrics.mean_squared_error(y_test, preds)
r2 = metrics.r2_score(y_test, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

Index(['Age Group', 'Type of Admission', 'Patient Disposition',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'Emergency Department Indicator'],
      dtype='object')
The model performance for testing set
--------------------------------------
MAE is 3.405481633936778
MSE is 44.10127623532677
R2 score is 0.1588617296710355


In [10]:
print(len(df_cleaned[df_cleaned['Length of Stay'] == '120 +']))
print(len(df_cleaned))

0
2308779
